# 7章 RNNによる文章生成

RNNとLSTMについて実装レベルで理解していれば，これを使ったアプリケーションの実装が可能．言語モデルの文章生成，seq2seqという新しい構造のニューラルネットワークを使用する．seq2seqは2つのRNNを組み合わせていとも簡単に実装可能．

## 7.1 言語モデルを使った文章生成

#### 言語モデルの汎用性
機械翻訳や音声認識，文章生成などがさまざまなアプリケーションにつかうことができる．そしてそれらは作成可能．

### 7.1.1 RNNによる文章生成の手順

#### 言語モデルは確率分布を出力する

#### 新たに次の単語を生成するには．．
最も高い単語を選ぶとか，確率的に選ぶとか，いろいろある．確率的に選ぶ場合は，あくまでも確率であり，決定的に一意に定まらないので，選ばれる単語は毎回異なる．
#### 決定的とは
- 決定的なアルゴリズム
- 確率的なアルゴリズム
#### そして機械は人間にとっても自然な文章を生成する
言語モデルは訓練データを丸暗記したものではない．そこで使われる単語の並び方のパターンを学習している．言語モデルがコーパスによって単語の出現パターンを正しく学習できているのなら，言語モデルが生成する文章は．人間にとっても自然な文章になることが期待できる．

### 7.1.2 文章生成の実装

単語ID，確率分布を用いて実際に文章生成を行う．ランダムな重み初期値では当然意味のない文章を生成してしまうが，学習済みの重みを使用すれば，ある程度意味のわかる部分が確認できる．コードの実装はvscodeでスクリプトを作成している．gitで確認できる．

### 7.1.3 さらに良い文章へ

学習後の重みをダウンロードし，正しくディレクトリに配置する．エディタでpickleファイルを見てみても相変わらずエンコード内容はわけがわからない．

## 7.2 seq2seq

#### 入出力が共に時系列データであるケースは世の中にたくさんある．
時系列データとして挙げられるのは，言語データ，音声データ，動画像データなどたくさんある．
#### seq2seq
時系列データを別の時系列データに変換するモデル．

### 7.2.1 seq2seqの原理

#### Encoder-Decoderモデル
EncoderとDecoderが登場する，seq2seqの別の呼び名．Encoderは入力データをエンコードし，Decoderはエンコードされたデータをデコードする．読んで字のごとくといった感じ．
#### エンコード
情報をある規則に基づいて変換すること
#### デコード
エンコードされた情報を元の情報に戻すこと

#### 例えばEncoderは．．
時系列データをhという隠れ状態ベクトルに変換する．

#### LSTMの隠れ状態hは固定長のベクトルである
要するに強調したいのは，エンコードとは任意の長さの文章を固定長のベクトルに変換するということである．

#### じゃあどう料理されてDecoderに？
それをここまで見てきた．例えば，Embedding, LSTM, Affine, Softmaxといったようなアーキテクチャ．

#### 唯一の小さな違い
LSTMがベクトルhを受け取るということ．ここまで見てきたモデルではLSTMレイヤは何も受け取っていない．

#### <code>eos</code>
Decoderに文章生成の開始を知らせる合図として利用される．

#### Encoder-Decoder間の架け橋
Encoder，Decoderにそれぞれある2つのLSTMレイヤの隠れ状態こそがEncoder-Decoderの間の架け橋である．

### 7.2.2 時系列データ変換用のトイ・プロブレム

#### トイ・プロブレム
機械学習を評価するために作られた簡単な問題．

### 7.2.3 可変長の時系列データ

#### 問題ごとに文字数が異なるなら．．
サンプルごとにデータの時間方向のサイズが異なるなら，それは可変長の時系列データを扱うことであり，ニューラルネットワークの学習におけるミニバッチ処理を行うために何らかの工夫が必要．

#### パディング
本来のデータを無効なデータで埋め，データの長さを均一に揃えるテクニック

#### 区切り文字
Decoderに文字列生成を知らせる合図として使われる．

#### パディングを用いるなら．．
本来存在しなかったパディング用の文字までseq2seqに処理させることになる．

### 7.2.4 足し算データセット

## 7.3 seq2seqの実装

#### seq2seqのざっくりしたアーキテクチャ

### 7.3.1 Encoderクラス

#### Encoderクラスのアーキテクチャ
RNNを用いてEncoderを実現する．EmbeddingレイヤとLSTMレイヤによって構成される．

#### LSTMレイヤは，右方向（時間方向）には隠れ状態とセルを出力．上方向には隠れ状態のみ出力．上方向にレイヤはないので破棄

#### 初期化メソッド
重みパラメータの初期化とレイヤの生成を行う．長い時系列データが一つだけ存在する問題として扱った際は<code>stateful=True</code>を指定して隠れ状態を維持したまま長い時系列データを処理した．今回は短い時系列データが複数存在する問題なので，問題ごとにLSTMの隠れ状態をリセットする必要がある．

#### メソッドの機能と理論
メソッドの詳細な挙動は，もうすでに学習済みとしてここでは説明を焼灼している．しかし．その機能の内容はただテキストを読んで一周しただけでは当然つかめない．

### 7.3.2 Decoderクラス

#### DecoderもEncoderと同様にRNNを使って実装する．

#### 学習時と生成時ではデータの与え方が異なる
学習時は正解がわかっているため，時系列方向のデータをまとめて与えることができる．一方推論時には，新しい文字列を生成するときには最初に開始を知らせる区切り文字を一つ与える．そしてその時の出力から文字を一つサンプリングし，それを次の入力にするという処理を繰り返し行う．

#### 決定的に文字列を生成させる．
最も高いスコアを持つ文字を一つ選ぶ．これまでに説明した確率的な考え方と対になる決定的な考え方のこと．

#### argmaxの追加
最大値を取るインデックス

#### softmax省略
softmaxは入力されたベクトルを正規化し，ベクトルの各要素の値は変換されるが．大小関係は変動がないためsoftmaxを省略し，Affineの出力をargmaxを適用する．

### 7.3.3 seq2seqクラス

#### EncoderクラスとDecoderクラスをつなぎ合わせ，TimeSoftmaxwithLossレイヤを使って損失を計算するだけ．

### 7.3.4 seq2seqの評価

#### 基本的なニューラルネットワークの学習を同じように学習が進められる
- 1. 学習データからミニバッチを選び
- 2. ミニバッチから勾配を計算し，
- 3. 勾配を使ってパラメータを更新する

## 7.4 seq2seqの改良

学習の進みを改善するための有望なテクニックはいくつか存在する．

### 7.4.1 入力データの反転（Reverse）

入力データを反転させるだけで，学習進度に大きく影響がある．理論的な根拠は説明できない．

### 7.4.2 覗き見（Peeky）

重要な情報を占有せずに，みんなで共有しようというもの．Affine，LSTMレイヤにEncoderの出力hを与える．

## 7.5 seq2seqを用いたアプリケーション

#### ある時系列データを別の時系列データに変換する
- 機械翻訳：ある言語の文章を別の言語の文章に変換する
- 自動要約：ある長い文章を短い要約された文章に変換する
- 質疑応答：質問を答えに変換する
- メールの自動返信：受け取ったメールの文章を返信文に変換する

### 7.5.1 チャットボット

### 7.5.2 アルゴリズムの学習

### 7.5.3 イメージキャプション

## 7.6 まとめ

#### 本章で学んだこと
- RNNを用いた言語モデルは新しい文章を生成することができる
- 文章生成を行う際には，ひとつの単語（もしくは文字）を与え，モデルの出力（確率分布）からサンプリングするという手順を繰り返し行う
- RNNを2つ組み合わせることで，時系列データを別の時系列データに変換することができる（seq2seq）
- seq2seqは，Encoderが入力文をエンコードし，そのエンコード情報をDecoderが受け取り，デコードして目的の出力文を得る
- 入力文を反転させること（Reverse），またエンコード情報をDecoderの複数のレイヤに与えること（Peeky）は，seq2seqの精度向上に有効である
- 機械翻訳やチャットボット，イメージキャプションなど，seq2seqはさまざまなアプリケーションに利用できる